In [1]:
import numpy as np
import pandas as pd
import requests
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ifeda\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ifeda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ifeda\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Parses and stores all 7 texts for preprocessing

In [52]:
url_list = [
    "https://raw.githubusercontent.com/formcept/whiteboard/master/nbviewer/notebooks/data/harrypotter/Book%201%20-%20The%20Philosopher's%20Stone.txt","https://raw.githubusercontent.com/formcept/whiteboard/master/nbviewer/notebooks/data/harrypotter/Book%202%20-%20The%20Chamber%20of%20Secrets.txt","https://raw.githubusercontent.com/formcept/whiteboard/master/nbviewer/notebooks/data/harrypotter/Book%203%20-%20The%20Prisoner%20of%20Azkaban.txt","https://raw.githubusercontent.com/formcept/whiteboard/master/nbviewer/notebooks/data/harrypotter/Book%204%20-%20The%20Goblet%20of%20Fire.txt","https://raw.githubusercontent.com/formcept/whiteboard/master/nbviewer/notebooks/data/harrypotter/Book%205%20-%20The%20Order%20of%20the%20Phoenix.txt",
    "https://raw.githubusercontent.com/formcept/whiteboard/master/nbviewer/notebooks/data/harrypotter/Book%206%20-%20The%20Half%20Blood%20Prince.txt",
    "https://raw.githubusercontent.com/formcept/whiteboard/master/nbviewer/notebooks/data/harrypotter/Book%207%20-%20The%20Deathly%20Hallows.txt"
    ]

text = []

for _,i in enumerate(url_list):
    req = requests.get(i)
    text.append(req.text)

## Preprocessing

In [83]:
def preprocessing(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)

    lower_filter = [w.lower() for w in text]
    filtered_text = []

    filtered_text = [i for i in filtered_text if not i.isdigit()]

    stop_words = stopwords.words('english') + [' j ','page','k','said','rowling','quot','back','mr','mrs']

    for words in lower_filter:
        if words not in stop_words:
            filtered_text.append(words)

    filtered_text = [i for i in filtered_text if not i.isdigit()]

    lem = WordNetLemmatizer()

    filtered_text = [lem.lemmatize(w) for w in filtered_text]

    return filtered_text

In [84]:
processed = [preprocessing(i) for _,i in enumerate(text)]

## Create Query Processor

In [94]:
def query_processing(text):
    tokenizer = RegexpTokenizer(r'\w+')
    processed_query = tokenizer.tokenize(text)

    processed_query = [w.lower() for w in processed_query]

    return processed_query

## Queries

In [131]:
queries = ["Who is the ghost that haunts the Hogwarts girls' bathroom?","Which animal is the Patronus of Severus Snape?","Who was the Defense Against the Dark Arts teacher in Harry's third year at Hogwarts","Who was the founder of Slytherin House and what was his gifts","What is the name of the spell that causes an object to rise and move according to the caster's will?"]

In [132]:
processed_queries = [query_processing(i) for _,i in enumerate(queries)]
processed_queries[0]

['who',
 'is',
 'the',
 'ghost',
 'that',
 'haunts',
 'the',
 'hogwarts',
 'girls',
 'bathroom']

## The Bag of Words representing the first book

In [133]:
processed[0][:10]

['boy',
 'lived',
 'dursley',
 'number',
 'four',
 'privet',
 'drive',
 'proud',
 'say',
 'perfectly']

## Prepare the Corpus and Indexing to form Doc-Term-Matrix

In [134]:
from gensim.corpora import Dictionary

X = Dictionary(processed)

doc_term_matrix = [X.doc2bow(doc) for doc in processed]

## Implementing the LSI Model

In [135]:
from gensim.models import LsiModel
lsi_model = LsiModel(corpus=doc_term_matrix,num_topics=7,id2word=X)

## 7 Key topics Corresponding with each book

In [136]:
lsi_model.print_topics()

[(0,
  '0.721*"harry" + 0.208*"ron" + 0.199*"potter" + 0.185*"hermione" + 0.159*"j" + 0.117*"dumbledore" + 0.098*"could" + 0.092*"know" + 0.090*"one" + 0.082*"like"'),
 (1,
  '0.488*"order" + 0.464*"phoenix" + -0.239*"harry" + 0.229*"umbridge" + 0.219*"sirius" + -0.176*"hallows" + -0.169*"deathly" + -0.130*"prince" + -0.122*"blood" + 0.121*"professor"'),
 (2,
  '0.403*"fire" + 0.389*"goblet" + -0.264*"hallows" + -0.254*"deathly" + -0.179*"blood" + -0.170*"prince" + -0.164*"half" + 0.152*"moody" + 0.147*"crouch" + -0.127*"dumbledore"'),
 (3,
  '-0.374*"prince" + -0.354*"blood" + 0.352*"hallows" + -0.348*"half" + 0.338*"deathly" + -0.242*"dumbledore" + -0.181*"slughorn" + 0.169*"hermione" + 0.166*"wand" + -0.141*"malfoy"'),
 (4,
  '0.300*"azkaban" + 0.292*"prisoner" + -0.273*"fire" + -0.268*"goblet" + -0.221*"dumbledore" + 0.204*"lupin" + 0.194*"black" + 0.171*"professor" + 0.153*"harry" + 0.138*"ron"'),
 (5,
  '-0.373*"chamber" + -0.338*"secret" + 0.324*"azkaban" + 0.302*"prisoner" + 0.

## Convert the Query into LSI Space

In [137]:
indexed_query = [Dictionary(processed).doc2bow(j) for j in processed_queries]
vector_query = [lsi_model[a] for a in indexed_query]
print(vector_query[0])

[(0, 0.03491504088121482), (1, -0.026611678787924493), (2, 0.028591680955725864), (3, -0.021295612037152935), (4, 0.009907197298940884), (5, -0.06193307475850284), (6, -0.007823154196609757)]


## Matrix Similarity

In [138]:
from gensim import similarities
index = similarities.MatrixSimilarity(lsi_model[doc_term_matrix])

## Perform Ranking

In [169]:
def doc_ranking_score(vector_query):
    similarities = index[vector_query]

    for i in similarities:
        t = np.argpartition(i,-4)[-4:]
        sol = i[t]
        sol = (list(t+1),list(sol[::-1]))
        print(f'The most relevant books in descending order are {sol[0]} with similarity scores of {sol[1]} ')

In [170]:
doc_ranking_score(vector_query)

The most relevant books in descending order are [6, 4, 2, 1] with similarity scores of [0.52452683, 0.6451366, 0.47689116, 0.44063973] 
The most relevant books in descending order are [7, 3, 6, 1] with similarity scores of [0.4560766, 0.49757794, 0.53445834, 0.41275486] 
The most relevant books in descending order are [6, 4, 3, 2] with similarity scores of [0.9622794, 0.9406679, 0.9330468, 0.9219502] 
The most relevant books in descending order are [5, 4, 2, 1] with similarity scores of [0.50057507, 0.5618626, 0.3176321, 0.2952755] 
The most relevant books in descending order are [6, 1, 4, 7] with similarity scores of [0.61576915, 0.53625244, 0.51299, 0.4980296] 


## Calculate key Metrics

## Create a BM25 Implementation for Comparison.....